In [8]:
usr = 'vokulovskiy'
pwd = 'OsPl27FLJC7c8N6t'
host = '212.24.35.176'
port = 10573

In [54]:
import psycopg2
import pandas as pd
import re

# pip install psycopg2-binary
# poetry add psycopg2-binary

min_price = 50000
usr = 'vokulovskiy'
pwd = 'OsPl27FLJC7c8N6t'
host = '212.24.35.176'
port = 10573

def search_keys(text, keys):
    '''
    Функция считает количество ключевых слов/фраз в тексте
    На входе - текст, список ключевых слов в виде списка фраз в формате regex
    На выходе - словарь с ключами и их количеством
    '''
    text =  text.lower().strip() # переводим в нижний регистр
    text = ' '.join(text.split()) # удаляем двойные пробелы
    cnt= {l[0]:len(l) for k in keys if len(l:=re.findall(k.lower(), text))>0}       
    return cnt

try:
    # пытаемся подключиться к базе данных
    conn = psycopg2.connect(dbname='attractor-raw', user=usr, password=pwd, host=host, port=port)
    print('Connected!')
except:
    # в случае сбоя подключения будет выведено сообщение  в STDOUT
    print('Can`t establish connection to database')

conn.autocommit = True  # устанавливаем актокоммит

df_db = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit=0 LIMIT 10", conn)
df_keys = pd.read_csv('key_words.csv',delimiter=';')
# Negative keys
keys_n = df_keys[(df_keys.np==0)].key.to_list()
# Positive keys
keys_p = df_keys[(df_keys.np==1)].key.to_list()

with conn.cursor() as curs:
    for idx,row in df_db.iterrows():
        if row.price < min_price:
            row.fit = -1
        else:
            text =  row.title.lower().strip() # переводим в нижний регистр
            count_n = len(search_keys(text, keys_n)) 
            count_p = len(search_keys(text, keys_p)) 
            if count_p==0 and count_n > 0:
                row.fit = -1
            else:
                row.fit = 1
        print(row.fit, row.price, row.title)
        sql = f"UPDATE gz44_ord SET fit = {row.fit} / 2 WHERE sk = {row.sk};"
        curs.execute(sql)
print('='*100)
df = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit!=0", conn)
print(df)
conn.close()

Connected!
-1 472437.0 Оказание услуг по предоставлению прав использования на условиях простой (неисключительной) лицензии программного обеспечения для нужд ФГБВУ "Центррегионводхоз"
-1 28036.8 УСЛУГИ ПО ПРЕДОСТАВЛЕНИЮ ЛИЦЕНЗИИ ПРОСТОЙ (НЕИСКЛЮЧИТЕЛЬНОЙ) НА ПРОГРАММНОЕ ОБЕСПЕЧЕНИЕ KASPERSKY
-1 337025.0 Оказание услуг по продлению неисключительных (пользовательских) прав на антивирусное программное обеспечение
-1 77000.0 Оказание услуг по поставке и передаче прав на использование программного обеспечения ViPNet Administrator 4.x (КС3)  и ViPNet Client 4.x (КС2) для защищенной сети № 2088
-1 87965.88 Оказание услуг по приобретению неисключительных (пользовательских) лицензионных прав (продление) на программное обеспечение
-1 568534.8 Оказание услуг по предоставлению лицензий на право использовать компьютерное программное обеспечение (средства криптографической защиты информации) для нужд ФГБУ «НМХЦ им. Н.И. Пирогова» Минздрава России


/tmp/ipykernel_1426462/3622125351.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit=0 LIMIT 10", conn)


-1 92666.6 Оказание услуг по продлению права использования ПО
-1 751600.0 Оказание услуг по предоставлению простого (неисключительного) права на получение обновлений, эксплуатируемого программного продукта «Медицинская Информационная Система «Комплекс Программных Средств «Система Автоматизации Медико-Страхового Обслуживания Населения» «САМСОН»
1 136800.0 оказание услуги по сопровождению компьютерного программного обеспечения «MedPOINT» в части пакета лицензий БАЗОВАЯ, СТАНДАРТ 60, ПОРТАЛ
-1 48394959.17 Оказание услуг по предоставлению неисключительных прав на средства защиты информации, мониторингу состояния информационной безопасности и обслуживанию средств защиты информации серверного сегмента Единой информационной системы здравоохранения Пермского края
Empty DataFrame
Columns: [sk, title, price, fit]
Index: []


/tmp/ipykernel_1426462/3622125351.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit!=0", conn)
